# Information Retrieval Evaluation Pipeline

This notebook implements a comprehensive evaluation pipeline for information retrieval systems using FAISS indices and BGE-M3 embeddings. The pipeline includes:
- Retrieval evaluation with multiple metrics
- Visualization of results
- Detailed performance analysis by categories
- Report generation

First, let's set up our environment and install required dependencies.

In [9]:
!pip install torch transformers sentence-transformers rank_bm25 nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
!pip install faiss-cpu FlagEmbedding tqdm numpy matplotlib seaborn

# from google.colab import drive
# drive.mount('/content/drive')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Import Required Libraries

Let's import all the necessary Python libraries and configure the environment.

In [1]:

import multiprocessing
import faiss
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
from FlagEmbedding import BGEM3FlagModel
from app.utils.retrievers.evaluator import RetrievalEvaluator
from app.utils.retrievers.multi_step import QueryRewriterRetrieverSubcategory
from app.utils.retrievers.retriever import Retriever
from langchain_openai import ChatOpenAI

/home/naif/miniconda3/envs/legal-assistant/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
faiss.omp_set_num_threads(12)

## Define Data Classes and Helper Functions

First, let's define our evaluation metrics dataclass and implement the core evaluation class.

## Visualization Functions

Let's implement functions to visualize our evaluation results.

## Run Evaluation Pipeline

Now let's run the evaluation pipeline. First, we need to:
1. Initialize the BGE-M3 model
2. Set up paths for the FAISS index and QA dataset
3. Create an evaluator instance
4. Run the evaluation
5. Generate visualizations and reports

In [5]:
# Configurations
retriever_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
metric_type = 'ip'
metric_suffix = 'V2' if metric_type == 'l2' else ''

index_path = 'data/m3_legal_faiss_brief.index'
qa_dataset_path ='data/evaluation_data/law_qa_dataset_validated.json'
documents_path = 'data/saudi_laws_scraped.json'
k_values=[1, 3, 5, 10, 20, 40, 80, 100]
# Load QA dataset
with open(qa_dataset_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    # We assume qa_pairs is a list of dicts, e.g.:
    # [{"query": "...", "relevant_ids": [123, 456]}, ...]
    qa_pairs = data.get('qa_pairs', [])
print(f"✓ Loaded {len(qa_pairs)} QA pairs")

print(f"Using {metric_type.upper()} metric with index: {index_path}")

retriever = Retriever(
    faiss_index_path=index_path,
    documents_path=documents_path,
    embeddings_model=retriever_model
)
llm = ChatOpenAI(
        openai_api_base ="http://localhost:11434/v1",
        api_key ="good",
        model="hf.co/unsloth/Qwen3-30B-A3B-Instruct-2507-GGUF:Q4_K_M",
        temperature=0,
        max_tokens=1000
    )
qw_retriever = QueryRewriterRetrieverSubcategory(
        llm=llm,
        faiss_index_path=index_path,
        documents_path=documents_path,
        embeddings_model=retriever_model,
)


evaluator_dense = RetrievalEvaluator(
    name="dense",
    approach_desc="Dense with IP Metric",
    model_name="BAAI/bge-m3",
    retrieve_function =retriever.dense,
    qa_pairs=qa_pairs,
    k_values=k_values,
    is_baseline=True,
)


evaluator_hybrid = RetrievalEvaluator(
        name="hybrid_dense_sparse",
        approach_desc="Hybrid (Dense + Sparse)",
        model_name="bge-small + BM25",
        retrieve_function =retriever.hybrid,
        qa_pairs=qa_pairs,
        k_values=k_values,
)

evaluator_qw_retriever = RetrievalEvaluator(
        name="",
        approach_desc="Query Rewriter+ filtering + Hybrid Retriever",
        model_name="Qwen3-bge-small + BM25",
        retrieve_function =qw_retriever.retrieve,
        qa_pairs=qa_pairs,
        k_values=k_values,
)



Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 87685.80it/s]


✓ Loaded 4156 QA pairs
Using IP metric with index: data/m3_legal_faiss_brief.index
Initializing Retriever...
Loading documents and metadata...
✓ Loaded 16371 documents with metadata
✓ BM25 initialized successfully!
Initializing Retriever...
Loading documents and metadata...
✓ Loaded 16371 documents with metadata


2025-10-25 20:05:21,766 - app.utils.retrievers.multi_step - INFO - ✓ Query Rewriter Retriever initialized


✓ BM25 initialized successfully!


In [4]:
reslts = evaluator_qw_retriever.evaluate_all()

Evaluating 280 queries for ''...


  0%|          | 0/280 [00:00<?, ?it/s]2025-10-25 18:41:29,569 - app.utils.retrievers.multi_step - INFO - Rewritten Query: تحليل شامل للنظام السياسي للدولة السعودية وفقًا للمفاهيم الموسعة، مع التركيز على الهيكل المؤسسي للدولة، وتفصيل السلطات التنفيذية والتشريعية والقضائية، والأنظمة المالية والرقابية، والأسس الدستورية والقانونية التي تُنظَم بها الحياة السياسية والمؤسساتية في المملكة العربية السعودية
2025-10-25 18:41:29,573 - app.utils.retrievers.multi_step - INFO - Relevant Terms: ['النظام السياسي', 'الهيكل المؤسسي', 'السلطات', 'الأنظمة المالية', 'الرقابة', 'الأسس الدستورية', 'الأسس القانونية', 'المملكة العربية السعودية']
2025-10-25 18:41:29,574 - app.utils.retrievers.multi_step - INFO - Subcategory Filters: ['الأنظمة الأساسية', 'السلطة القضائية وحقوق الإنسان', 'الخدمة المدنية', 'المال والرقابة', 'التنظيمات الإدارية']
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text foll

In [5]:
evaluator_qw_retriever.save_results('.')

✓ Saved plot to ./evaluation_metrics_.png
✓ Saved all results for '' to .


In [6]:
from app.utils.retrievers.evaluator import compare_approaches

compare_approaches([evaluator_qw_retriever], metric_type="ex")


## METRIC COMPARISON: EX | BASELINE: Query Rewriter+ filtering + Hybrid Retriever

| Metric        | Query Rewriter+ filtering + Hybrid Retriever |
| -------------- | --------------------------------------------- |
| Recall@1      | 0.0040                                       |
| Recall@3      | 0.0065                                       |
| Recall@5      | 0.0081                                       |
| Recall@10     | 0.0133                                       |
| Recall@20     | 0.0264                                       |
| Recall@40     | 0.0390                                       |
| Recall@80     | 0.0621                                       |
| Recall@100    | 0.0720                                       |
| Precision@1   | 0.0179                                       |
| Precision@3   | 0.0107                                       |
| Precision@5   | 0.0079                                       |
| Precision@10  | 0.0071                                       |
| Pr

In [6]:
from app.utils.retrievers.evaluator import compare_approaches
approaches = [evaluator_dense, evaluator_hybrid]
EVAL_DIR = 'evaluation_pipeline/assets'
METRIC_TYPE = 'retrieval'
METRIC_SUFFIX = "relational_V0"
# 3. Run evaluations
print(f"\nStarting evaluation for {len(approaches)} approaches...")
for approach in approaches:
    approach.evaluate_all()


# 4. Save results (plot, report, details)
print("\nGenerating visualizations and saving results...")
for approach in approaches:
    approach.save_results(
        evaluation_dir=EVAL_DIR,
        metric_suffix=METRIC_SUFFIX,
        # METRIC_TYPE=METRIC_TYPE
    )

print("\n✓ Evaluation workflow completed successfully!")
# 5. Compare all approaches
compare_approaches(
    approaches,
    METRIC_TYPE
)


Starting evaluation for 2 approaches...
Evaluating 4156 queries for 'dense'...


100%|██████████| 4156/4156 [01:38<00:00, 42.38it/s]


Evaluating 4156 queries for 'hybrid_dense_sparse'...


100%|██████████| 4156/4156 [01:41<00:00, 40.93it/s]



Generating visualizations and saving results...
✓ Saved plot to evaluation_pipeline/assets/evaluation_metrics_dense_relational_V0.png
✓ Saved all results for 'dense' to evaluation_pipeline/assets
✓ Saved plot to evaluation_pipeline/assets/evaluation_metrics_hybrid_dense_sparse_relational_V0.png
✓ Saved all results for 'hybrid_dense_sparse' to evaluation_pipeline/assets

✓ Evaluation workflow completed successfully!

## METRIC COMPARISON: RETRIEVAL | BASELINE: Dense with IP Metric

| Metric        | Dense with IP Metric | Hybrid (Dense + Sparse) |
| -------------- | --------------------- | ------------------------ |
| Recall@1      | 0.4645               | 0.5235 (+12.7%)         |
| Recall@3      | 0.6237               | 0.6825 (+9.4%)          |
| Recall@5      | 0.6752               | 0.7335 (+8.6%)          |
| Recall@10     | 0.7378               | 0.7843 (+6.3%)          |
| Recall@20     | 0.7940               | 0.8324 (+4.8%)          |
| Recall@40     | 0.8425               | 